In [1]:
import sys
sys.path.append("../../")
from BDA.stats import *
from BDA.plots import *

now importing: 
1. numpy as np 
2. from scipy import stats


now importing: 
3. matplotlib.pyplot as plt



Survival analysis is the data analysis of time to event data such as death/failure. This type of data can be (i) time until death after diagnosis of disease, (ii) time until a set of lightbulbs burn out.

The analysis of this data focuses on summarizing the main features of the distribution, such as median and other percentiles of time to failure, and on examining the effects of explanatory variables. Data on times until failure, or more optimistically, duration of survival or survival times, have two important features:

a. times are positive (non-negative) and typically have skewed distributions with long tails (think exponential distribution)

b. some of the subjects may survive beyond the study period (or period of observation). Some subjects may drop-out before we are able to observe how long they survive; we may only partially observe these subjects

c. Another type of censoring is when a person's true survival time is less than what we observed. Say a person caught a disease months before actually diagnosed.

Points b and c refer to left- and right-censoring. Censoring complicates all of the technical issues involved in analyzing the data. 

In 

In [5]:
duration = rexp(1000)
# help(rexp)

In [6]:
hist(duration, bins=20, title="Survival Times", xlabel="time", ylabel = "Count", density=False)

In [7]:
help(rexp)

Help on function rexp in module BDA.stats:

rexp(n, rate=None, loc=0, scale=1, log=False)



Since survival data are characterised by positive values and skewed distributions with long tails, we'll be looking at the Exponential, Weibull or Gamma distributions for our data models.

A few things to consider:

- time will refer to minutes, hours, days, years

- event can refer to death, disease incidence, or any other event of interest

- multiple events can be analyzed simultaneously; these types of models are refered to as competing risk models, but we will not delve into these models

In survival analysis we usually refer to the time variable as survival time, because it gives the time that an individual has "survived" over some follow-up period. We also, typically refer to the event as a failure because the event of interst is usually some negative event like death.


Most survival analyses must consider handling censored data. Essentially, for some groups of observations, we may not know the exact survival time.

In our notation, we will denote the random variable with a capital *T* and the observed/specific time value with a lower case *t*.

We denote the small letter d, $d \in (0,1)$, as a random variable indicating censorship. That is, if $d = 1$, then the observation was censored (i.e. only partially observed).

A focus for suvival analysis is in estimating the survivor function, $S(t)$, and the hazard function, $h(t)$.

If we are talking about people, the survivor function is simply the probability that a person survives longer than some time $t$.

$$
S(t) = P(T > t) = 1 - F(t)
$$

and the hazard function is a rate function, defined as 

$$
h(t) = \lim_{\delta t \to 0}  \frac{P(t \leq T < t + \delta t | T \geq t)}{\delta t}
$$

The relationship between the survival function and hazard function can be written as

$$
S(t) = exp\left[ - \int_0^t h(u) du \right]
$$

or

$$
h(t) = - \left[ \frac{d S(t)}{S(t)} \right]
$$


The first formula describes how the survivor function equals the exponential of the negative integral of the hazard function between integration limits of 0 and t.

The second describes how the hazard function equals negative the derivative of $S(t)$ with respec to t divided by $S(t)$.

Thus, if you know either the survivor function or hazard function, you can estimate the other.